In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
import itertools
import numpy as np
import scipy.sparse as sp

Using backend: pytorch[17:41:02] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /mnt/berry/home/prakhar/dgl/dgl/lib/python3.8/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.9.1.so: cannot open shared object file: No such file or directory



In [2]:
import dgl.data
import urllib.request
import pandas as pd

urllib.request.urlretrieve("https://data.dgl.ai/tutorial/dataset/members.csv","./members.csv")
members=pd.read_csv("./members.csv")
urllib.request.urlretrieve("https://data.dgl.ai/tutorial/dataset/interactions.csv","./interactions.csv")
interactions=pd.read_csv("./interactions.csv")
print(members.head())
print(interactions.head())

   Id    Club  Age
0   0  Mr. Hi   44
1   1  Mr. Hi   37
2   2  Mr. Hi   37
3   3  Mr. Hi   40
4   4  Mr. Hi   30
   Src  Dst    Weight
0    0    1  0.043591
1    0    2  0.282119
2    0    3  0.370293
3    0    4  0.730570
4    0    5  0.821187


In [8]:
import dgl
from dgl.data import DGLDataset
import torch
import os

class KarateClubDataset(DGLDataset):
    def __init__(self):
        super().__init__(name="karate_club")
    
    def process(self):
        nodes_data=pd.read_csv("./members.csv")
        edges_data=pd.read_csv("./interactions.csv")
        node_features=torch.from_numpy(nodes_data["Age"].to_numpy())
        edge_features=torch.from_numpy(edges_data["Weight"].to_numpy())
        node_labels=torch.from_numpy(nodes_data["Club"].astype("category").cat.codes.to_numpy())
        edge_src=torch.from_numpy(edges_data["Src"].to_numpy())
        edge_dst=torch.from_numpy(edges_data["Dst"].to_numpy())
        
        self.graph=dgl.graph((edge_src,edge_dst),num_nodes=nodes_data.shape[0])
        self.graph.ndata["feat"]=node_features
        self.graph.ndata["label"]=node_labels
        self.graph.edata["weight"]=edge_features
        
        
        n_nodes=nodes_data.shape[0]
        n_train=int(n_nodes*0.6)
        n_val=int(n_nodes*2)
        train_mask=torch.zeros(n_nodes,dtype=torch.bool)
        val_mask=torch.zeros(n_nodes,dtype=torch.bool)
        test_mask=torch.zeros(n_nodes,dtype=torch.bool)
        train_mask[:n_train]=True
        val_mask[n_train:n_train+n_val]=True
        test_mask[n_train+n_val:]=True
        self.graph.ndata["train_mask"]=train_mask
        self.graph.ndata["val_mask"]=val_mask
        self.graph.ndata["test_mask"]=test_mask
        
    def __getitem__(self,i):
        return self.graph

    def __len__(self):
        return 1
    
dataset=KarateClubDataset()
print(dataset)
graph=dataset[0]
print(graph)

Graph(num_nodes=34, num_edges=156,
      ndata_schemes={'feat': Scheme(shape=(), dtype=torch.int64), 'label': Scheme(shape=(), dtype=torch.int8), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})


DGLError: Invalid key "0". Must be one of the edge types.

In [63]:
print("node features",g.ndata)
print((g.ndata["feat"].shape))
print((g.ndata["label"].unique().size())[0])
labels=g.ndata["label"]
train_mask=g.ndata["train_mask"]
print(labels[train_mask].shape)

node features {'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False])}
torch.Size([2708, 1433])
7
torch.Size([140])


In [23]:
print("edge features",g.edata)

edge features {}


In [59]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self,in_feat,h_feat, num_classes):
        super(GCN,self).__init__()
        self.conv1=GraphConv(in_feat,h_feat)
        self.conv2=GraphConv(h_feat,num_classes)
    def forward(self,g,in_feat):
        h=self.conv1(g,in_feat)
        h=F.relu(h)
        h=self.conv2(g,h)
        return h
    
model=GCN(g.ndata["feat"].shape[1],16,g.ndata["label"].unique().size()[0])

In [71]:
def train(g,model):
    optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
    best_val_acc,best_test_acc=0,0
    features=g.ndata["feat"]
    lables=g.ndata["label"]
    train_mask=g.ndata["train_mask"]
    val_mask=g.ndata["val_mask"]
    test_mask=g.ndata["test_mask"]
    
    for e in range(200):
#         forward
        logits=model(g,features)
#         prediction
        preds=logits.argmax(1)
        loss=F.cross_entropy(logits[train_mask],labels[train_mask])
        train_acc=(preds[train_mask]==labels[train_mask]).float().mean()
        val_acc=(preds[val_mask]==labels[val_mask]).float().mean()
        test_acc=(preds[test_mask]==labels[test_mask]).float().mean()
        
        if best_val_acc<val_acc:
            best_val_acc=val_acc
            best_test_acc=test_acc
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if e%5==0:
            print("In epoch{}, loss: {: .3f}, val_acc: {: .3f} (best: {: .3f}), test_acc: {: .3f} (best test: {: .3f})".format(e,loss,val_acc,best_val_acc,test_acc,best_test_acc))
        
model=GCN(g.ndata["feat"].shape[1],16,g.ndata["label"].unique().size()[0])
train(g,model)

In epoch0, loss:  1.946, val_acc:  0.118 (best:  0.118), test_acc:  0.116 (best test:  0.116)
In epoch5, loss:  1.900, val_acc:  0.522 (best:  0.522), test_acc:  0.547 (best test:  0.547)
In epoch10, loss:  1.829, val_acc:  0.658 (best:  0.658), test_acc:  0.691 (best test:  0.691)
In epoch15, loss:  1.736, val_acc:  0.648 (best:  0.658), test_acc:  0.689 (best test:  0.691)
In epoch20, loss:  1.617, val_acc:  0.666 (best:  0.666), test_acc:  0.700 (best test:  0.700)
In epoch25, loss:  1.476, val_acc:  0.706 (best:  0.706), test_acc:  0.731 (best test:  0.731)
In epoch30, loss:  1.316, val_acc:  0.722 (best:  0.724), test_acc:  0.747 (best test:  0.745)
In epoch35, loss:  1.143, val_acc:  0.736 (best:  0.736), test_acc:  0.759 (best test:  0.759)
In epoch40, loss:  0.967, val_acc:  0.742 (best:  0.742), test_acc:  0.762 (best test:  0.760)
In epoch45, loss:  0.799, val_acc:  0.750 (best:  0.750), test_acc:  0.768 (best test:  0.768)
In epoch50, loss:  0.648, val_acc:  0.754 (best:  0.